# MoM implementation in Julia
# Initiated by Lisette 
# Editted by Domenico in Attempt to Render Implementation Type-Stable 

## Import Packages 

In [1]:
using LinearAlgebra
using StructArrays 
using Statistics
using StaticArrays
#using PhysicalConstants.CODATA2018
using BenchmarkTools

In [2]:
# a point in 3D is a tuple of 3 coordinates 
# although not strictly necessary, we here introduce static vectors
# static vectors might be valuable to take advantage of in a later stage
const Point3D = SVector{3,Float64};

## Section 1: Introduction 
1. method of moments as in Morandi;
2. magnetization vector assumed constant per element - shape functions are element-wise constant;
3. collaction or point matching - test function are Dirac delta distributions; 
4. more later; 

## Section 2: Space for Experimentation 

I just wanted to add, maybe you will find StructArrays.jl useful. It lets you deal with arrays-of-structs, while internally being a struct-of-arrays. Used in conjunction with Accessors.jl, this can be pretty handy. See this answer.

In [3]:
# What follows is UGLY, but can be fixed later. We desire to show a proof of concept first. 
    
# Vertices holds the eight (8) vertices of a face 
struct Vertices 
    vertex1::Point3D
    vertex2::Point3D
    vertex3::Point3D
    vertex4::Point3D
    vertex5::Point3D
    vertex6::Point3D
    vertex7::Point3D
    vertex8::Point3D
end 

struct Triangle
    triangle1::Point3D
    triangle2::Point3D 
    triangle3::Point3D 
end 

struct Faces  
    face1::Triangle
    face2::Triangle
    face3::Triangle
    face4::Triangle
    face5::Triangle
    face6::Triangle
    face7::Triangle
    face8::Triangle
    face9::Triangle
    face10::Triangle
    face11::Triangle
    face12::Triangle    
end 

In [17]:
# vertices only - good for testing 
function gen_multi_box_vertices(length::Float64, width::Float64, height::Float64, 
         num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64)

    num_boxes = num_boxes_x * num_boxes_y * num_boxes_z 
    vertices_list = StructArray{Vertices}(undef,num_boxes)
    count_boxes = 0; 
    for i in 1:num_boxes_x
        for j in 1:num_boxes_y
            for k in 1:num_boxes_z
                x_start = (i - 1) * length
                x_end = i * length
                y_start = (j - 1) * width
                y_end = j * width
                z_start = (k - 1) * height
                z_end = k * height

                vertices =  StructArray{Vertices}((
                    Point3D(x_start, y_start, z_start),
                    Point3D(x_end, y_start, z_start),
                    Point3D(x_end, y_end, z_start),
                    Point3D(x_start, y_end, z_start),
                    Point3D(x_start, y_start, z_end),
                    Point3D(x_end, y_start, z_end),
                    Point3D(x_end, y_end, z_end),
                    Point3D(x_start, y_end, z_end)
                )) 

                count_boxes += 1 
                vertices_list[count_boxes] = vertices
                                
            end
        end
    end

    return vertices_list
end

gen_multi_box_vertices (generic function with 1 method)

In [18]:
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

LoadError: MethodError: [0mCannot `convert` an object of type [92mStructVector{Vertices, NamedTuple{(:vertex1, :vertex2, :vertex3, :vertex4, :vertex5, :vertex6, :vertex7, :vertex8), NTuple{8, SVector{3, Float64}}}, Int64}[39m[0m to an object of type [91mVertices[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T at Base.jl:61
[0m  Vertices(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at In[3]:5

In [6]:
length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1 

# absence of outside in red fontr means code is ok 
# code in orange font means that matter to look into remain 
@code_warntype gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

MethodInstance for gen_multi_box_vertices(::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64)
  from gen_multi_box_vertices(length::Float64, width::Float64, height::Float64, num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64) in Main at In[5]:2
Arguments
  #self#::Core.Const(gen_multi_box_vertices)
  length::Float64
  width::Float64
  height::Float64
  num_boxes_x::Int64
  num_boxes_y::Int64
  num_boxes_z::Int64
Locals
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  count_boxes::Int64
  vertices_list::Vector{Vertices}
  num_boxes::Int64
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
  @_14::Union{Nothing, Tuple{Int64, Int64}}
  j::Int64
  k::Int64
  vertices::StructVector{Vertices, NamedTuple{(:vertex1, :vertex2, :vertex3, :vertex4, :vertex5, :vertex6, :vertex7, :vertex8), NTuple{8, SVector{3, Float64}}}, Int64}
  z_end::Float64
  z_start::Float64
  y_end::Float64
  y_start::Float64
  x_end::Float64
  x_start::Float64
Body::Vector{Vertices}
1 ─       (num_boxes = n

In [7]:
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

num_boxes_x = 100; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

LoadError: MethodError: [0mCannot `convert` an object of type [92mStructVector{Vertices, NamedTuple{(:vertex1, :vertex2, :vertex3, :vertex4, :vertex5, :vertex6, :vertex7, :vertex8), NTuple{8, SVector{3, Float64}}}, Int64}[39m[0m to an object of type [91mVertices[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T at Base.jl:61
[0m  Vertices(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at In[3]:5

In [8]:
function generate_multi_box(length::Float64, width::Float64, height::Float64, 
         num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64)

    num_boxes = num_boxes_x * num_boxes_y * num_boxes_z 
    vertices_list = Array{Vertices,1}(undef,num_boxes)
    faces_list = Array{Faces,1}(undef,num_boxes)
    count_boxes = 0; 
    for i in 1:num_boxes_x
        for j in 1:num_boxes_y
            for k in 1:num_boxes_z
                x_start = (i - 1) * length
                x_end = i * length
                y_start = (j - 1) * width
                y_end = j * width
                z_start = (k - 1) * height
                z_end = k * height

                vertices =  Vertices(
                    Point3D(x_start, y_start, z_start),
                    Point3D(x_end, y_start, z_start),
                    Point3D(x_end, y_end, z_start),
                    Point3D(x_start, y_end, z_start),
                    Point3D(x_start, y_start, z_end),
                    Point3D(x_end, y_start, z_end),
                    Point3D(x_end, y_end, z_end),
                    Point3D(x_start, y_end, z_end)
                )

                count_boxes += 1 
                vertices_list[count_boxes] = vertices
                
                faces = Faces(
                    Triangle(vertices[1], vertices[2], vertices[3]),
                    Triangle(vertices[1], vertices[3], vertices[4]),

                    Triangle(vertices[5], vertices[6], vertices[7]),
                    Triangle(vertices[5], vertices[7], vertices[8]),

                    Triangle(vertices[1], vertices[2], vertices[6]),
                    Triangle(vertices[1], vertices[6], vertices[5]),

                    Triangle(vertices[3], vertices[4], vertices[8]),
                    Triangle(vertices[3], vertices[8], vertices[7]),

                    Triangle(vertices[2], vertices[3], vertices[7]),
                    Triangle(vertices[2], vertices[7], vertices[6]),

                    Triangle(vertices[1], vertices[4], vertices[8]),
                    Triangle(vertices[1], vertices[8], vertices[5]) 
                ) 

                faces_list[count_boxes] = faces 
                
            end
        end
    end

    return vertices_list, faces_list
end

generate_multi_box (generic function with 1 method)

In [9]:
length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1 

# absence of outside in red fontr means code is ok 
# code in orange font means that matter to look into remain 
@code_warntype generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

MethodInstance for generate_multi_box(::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64)
  from generate_multi_box(length::Float64, width::Float64, height::Float64, num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64) in Main at In[8]:1
Arguments
  #self#::Core.Const(generate_multi_box)
  length::Float64
  width::Float64
  height::Float64
  num_boxes_x::Int64
  num_boxes_y::Int64
  num_boxes_z::Int64
Locals
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  count_boxes::Int64
  faces_list::Vector{Faces}
  vertices_list::Vector{Vertices}
  num_boxes::Int64
  @_13::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
  @_15::Union{Nothing, Tuple{Int64, Int64}}
  j::Int64
  k::Int64
  faces::Union{}
  vertices::Vertices
  z_end::Float64
  z_start::Float64
  y_end::Float64
  y_start::Float64
  x_end::Float64
  x_start::Float64
Body::Tuple{Vector{Vertices}, Vector{Faces}}
1 ─        (num_boxes = num_boxes_x * num_boxes_y * num_boxes_z)
│   %2   = Core.apply_type(Main.Array, Main.Vertice

In [10]:
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
@btime generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
@btime generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

num_boxes_x = 100; num_boxes_y = 1; num_boxes_z = 1
@btime generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

LoadError: MethodError: no method matching getindex(::Vertices, ::Int64)

## Section 3: Mesh Generation 
We first define 3 user defined types, namely Point3D, Vertices and Faces  

In [35]:
# Vertices holds the eight (8) vertices of a face 
const Vertices = Array{Point3D,1};

# Faces holds the six (6) faces of a block 
const Faces = Array{Vertices,1};

In [36]:
# show use case 
xstart = 0.; xend = 1.; ystart = 0.; yend = 1.; zstart = 0.; zend = 1.;   
point = Point3D(xstart,ystart,zstart)
# show what function typeof() does
typeof(point)

SVector{3, Float64} (alias for SArray{Tuple{3}, Float64, 1, 3})

What follows requires further editting to make the user defined types to work properly. 

In [34]:
function generate_multi_box(length::Float64, width::Float64, height::Float64, 
         num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64)

    num_boxes = num_boxes_x * num_boxes_y * num_boxes_z 
    vertices_list = Array{Vertices,1}(undef,num_boxes)
    faces_list = Array{Faces,1}(undef,num_boxes)
    count_boxes = 0; 
    for i in 1:num_boxes_x
        for j in 1:num_boxes_y
            for k in 1:num_boxes_z
                x_start = (i - 1) * length
                x_end = i * length
                y_start = (j - 1) * width
                y_end = j * width
                z_start = (k - 1) * height
                z_end = k * height

                vertices =  [
                    Point3D(x_start, y_start, z_start),
                    Point3D(x_end, y_start, z_start),
                    Point3D(x_end, y_end, z_start),
                    Point3D(x_start, y_end, z_start),
                    Point3D(x_start, y_start, z_end),
                    Point3D(x_end, y_start, z_end),
                    Point3D(x_end, y_end, z_end),
                    Point3D(x_start, y_end, z_end)
                ]

                count_boxes += 1 
                vertices_list[count_boxes] = vertices
                
                faces = [
                    [vertices[1], vertices[2], vertices[3]],
                    [vertices[1], vertices[3], vertices[4]],

                    [vertices[5], vertices[6], vertices[7]],
                    [vertices[5], vertices[7], vertices[8]],

                    [vertices[1], vertices[2], vertices[6]],
                    [vertices[1], vertices[6], vertices[5]],

                    [vertices[3], vertices[4], vertices[8]],
                    [vertices[3], vertices[8], vertices[7]],

                    [vertices[2], vertices[3], vertices[7]],
                    [vertices[2], vertices[7], vertices[6]],

                    [vertices[1], vertices[4], vertices[8]],
                    [vertices[1], vertices[8], vertices[5]]
                ]

                faces_list[count_boxes] = faces 
                
            end
        end
    end

    return vertices_list, faces_list
end


generate_multi_box (generic function with 1 method)

In [12]:
length_box = 1.0 
width = 1.0
height = 1.0

num_boxes_x = 2
num_boxes_y = 1
num_boxes_z = 1

vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

display(vertices_list[2])

display(faces_list[2])

8-element Vector{SVector{3, Float64}}:
 [1.0, 0.0, 0.0]
 [2.0, 0.0, 0.0]
 [2.0, 1.0, 0.0]
 [1.0, 1.0, 0.0]
 [1.0, 0.0, 1.0]
 [2.0, 0.0, 1.0]
 [2.0, 1.0, 1.0]
 [1.0, 1.0, 1.0]

12-element Vector{Vector{SVector{3, Float64}}}:
 [[1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [2.0, 1.0, 0.0]]
 [[1.0, 0.0, 0.0], [2.0, 1.0, 0.0], [1.0, 1.0, 0.0]]
 [[1.0, 0.0, 1.0], [2.0, 0.0, 1.0], [2.0, 1.0, 1.0]]
 [[1.0, 0.0, 1.0], [2.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
 [[1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [2.0, 0.0, 1.0]]
 [[1.0, 0.0, 0.0], [2.0, 0.0, 1.0], [1.0, 0.0, 1.0]]
 [[2.0, 1.0, 0.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]]
 [[2.0, 1.0, 0.0], [1.0, 1.0, 1.0], [2.0, 1.0, 1.0]]
 [[2.0, 0.0, 0.0], [2.0, 1.0, 0.0], [2.0, 1.0, 1.0]]
 [[2.0, 0.0, 0.0], [2.0, 1.0, 1.0], [2.0, 0.0, 1.0]]
 [[1.0, 0.0, 0.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]]
 [[1.0, 0.0, 0.0], [1.0, 1.0, 1.0], [1.0, 0.0, 1.0]]

In [37]:
num_boxes_x = 2
num_boxes_y = 1
num_boxes_z = 1 

# absence of outside in red fontr means code is ok 
# code in orange font means that matter to look into remain 
@code_warntype generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

MethodInstance for generate_multi_box(::Float64, ::Float64, ::Float64, ::Int64, ::Int64, ::Int64)
  from generate_multi_box(length::Float64, width::Float64, height::Float64, num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64) in Main at In[34]:1
Arguments
  #self#::Core.Const(generate_multi_box)
  length::Float64
  width::Float64
  height::Float64
  num_boxes_x::Int64
  num_boxes_y::Int64
  num_boxes_z::Int64
Locals
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  count_boxes::Int64
  faces_list::Vector{Vector{Vector{SVector{3, Float64}}}}
  vertices_list::Vector{Vector{SVector{3, Float64}}}
  num_boxes::Int64
  @_13::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
  @_15::Union{Nothing, Tuple{Int64, Int64}}
  j::Int64
  k::Int64
  faces::Vector{Vector{SVector{3, Float64}}}
  vertices::Vector{SVector{3, Float64}}
  z_end::Float64
  z_start::Float64
  y_end::Float64
  y_start::Float64
  x_end::Float64
  x_start::Float64
Body::Tuple{Vector{Vector{SVector{3, Float64}}}, Vector{Vector{V

In [39]:
# the number of allocations remains to high! 
num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
@btime generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
@btime generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

  683.775 ns (30 allocations: 3.91 KiB)
  3.203 μs (142 allocations: 19.16 KiB)


## Section 4: Utility Functions 

In [44]:
function calculate_normal(face_vertices)
    u = face_vertices[2] - face_vertices[1]
    v = face_vertices[3] - face_vertices[1]

    normal_vector = cross(u, v)
    normal_unit_vector = normal_vector / norm(normal_vector)

    return normal_unit_vector
end

function calculate_normal!(result,face_vertices)
    u = face_vertices[2] - face_vertices[1]
    v = face_vertices[3] - face_vertices[1]

    result = myCross!(result,u, v)
    result = result / norm(result)

    return result 
end

function find_center_r_k(vertices)
    return sum(vertices) / length(vertices)
end

function r_norm(r_1, r_2, epsilon)
    return sqrt(norm(r_1 - r_2)^2 + epsilon^2)
end

function find_w_e(r_1, r_2, r_k, epsilon)
    return log((r_norm(r_2, r_k, epsilon) + r_norm(r_1, r_k, epsilon) + r_norm(r_2, r_1, epsilon)) / (r_norm(r_2, r_k, epsilon) + r_norm(r_1, r_k, epsilon) - r_norm(r_2, r_1, epsilon)))
end

function solid_angle_triangular_plane(r_1, r_2, r_3, r_k, epsilon)
    D_r = r_norm(r_1, r_k, epsilon) * r_norm(r_2, r_k, epsilon) * r_norm(r_3, r_k, epsilon) + r_norm(r_1, r_k, epsilon) * dot((r_2 - r_k), (r_3 - r_k)) + r_norm(r_2, r_k, epsilon) * dot((r_1 - r_k), (r_3 - r_k)) + r_norm(r_3, r_k, epsilon) * dot((r_1 - r_k), (r_2 - r_k))
    x1 = dot((r_1 - r_k), SVector{3}(cross(SVector{3}((r_2 - r_k)), SVector{3}(((r_3 - r_k))))))
    return 2 * atan(x1, D_r)
end

function myCross!(result,a,b)
# computes outer product of a and b and stores results in w
# assumption is that memory for w has been allocated outside this function
# not sure whether this is necessary at all     
    result[1] = a[2]*b[3] - a[3]*b[2]
    result[2] = -a[1]*b[3] + a[3]*b[1]
    result[3] = a[1]*b[3] - a[3]*b[1]
    return result
end 

myCross! (generic function with 1 method)

## Section 4: Code by Lisette to Compute the Matrix 

In [ ]:
function orig_build_C_matrix(vertices_list, faces_list)
    N = length(vertices_list)
    eps = 0.001
    I_3 = Matrix{Float64}(I, 3, 3)
    C = zeros(3 * N, 3 * N)


    for (ele, vertex_set) in enumerate(vertices_list)
        r_k = find_center_r_k(vertex_set)

        for (source, face_set) in enumerate(faces_list)
            
            #C_subblock = SMatrix{3, 3}(zeros(3, 3))
            C_subblock = zeros(3, 3)
            for (S_f, face) in enumerate(face_set)
                
                # Calculate the normal vector on the plane
                n_f = calculate_normal(face)

                # Calculate center of the face
                r_k_face = find_center_r_k(vertices_list[source])

                # Calculate vector from vertex to center
                centroid_face = mean(face, dims=1)
                
                if dot(n_f, r_k_face - centroid_face[1]) > 0
                    n_f = -n_f
                    face = [face[1], face[3], face[2]]
                end

                r_1 = face[1]
                r_2 = face[2]
                r_3 = face[3]

                solid_angle = solid_angle_triangular_plane(r_1, r_2, r_3, r_k, eps)

                t_e_w_e = zeros(Float64,3,)
                for (v, vertex) in enumerate(face)
                    v_next = mod(v, 3) + 1
                    current_edge = face[v_next] - face[v]

                    t_e = current_edge / norm(current_edge)
                    w_e = find_w_e(face[v], face[v_next], r_k, eps)
                    t_e_w_e = t_e_w_e + w_e* t_e
                end

                gradient_W = cross(n_f, cross(t_e_w_e, n_f)) + n_f * solid_angle
                
                for i in 1:size(I_3, 1)
                    u_i = I_3[:, i]
                    u_i_transposed = reshape(u_i, 1, :)
                    #C_subblock -= SMatrix{3,3}((1.25663706212e-6/ (4 * pi)) * cross(cross(u_i, n_f), gradient_W) * u_i_transposed)
                    C_subblock -= ((1.25663706212e-6/ (4 * pi)) * cross(cross(u_i, n_f), gradient_W) * u_i_transposed)
                    
                end
            end

            C[3 * (ele - 1) + 1:3 * ele, 3 * (source - 1) + 1:3 * source] = C_subblock
        end
    end

    return C
end

In [48]:
length_box = 1.0
width = 1.0
height = 1.0

num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = orig_build_C_matrix(vertices_list, faces_list)

  53.083 μs (1846 allocations: 143.98 KiB)


6×6 Matrix{Float64}:
 8.37757e-7   0.0          0.0         1.69372e-7   0.0          0.0
 0.0          8.37757e-7   0.0         0.0         -8.46861e-8  -1.38919e-7
 0.0          0.0          8.37757e-7  0.0          1.38919e-7  -8.46861e-8
 1.69372e-7   0.0          0.0         8.37757e-7   0.0          0.0
 0.0         -8.46861e-8   1.38919e-7  0.0          8.37757e-7   0.0
 0.0         -1.38919e-7  -8.46861e-8  0.0          0.0          8.37757e-7

## Section 6: Sandbox for Domenico to Play in 
The function blank_C_matrix() allocates too much. 

In [40]:
length_box = 1.
width = 1.
height = 1.

num_boxes_x = 2
num_boxes_y = 1
num_boxes_z = 1

vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

In [45]:
function blank_C_matrix(vertices_list, faces_list)
    N = length(vertices_list)
    eps = 0.001
    I_3 = Matrix{Float64}(I, 3, 3)
    C = zeros(3 * N, 3 * N)
    # added by domenico 
    C_subblock = zeros(Float64,3,3) 
    nf = zeros(3)
    w1 = zeros(3) # first temporary vector of size 3 
    w2 = zeros(3) # first temporary vector of size 3 

    for (ele, vertex_set) in enumerate(vertices_list)

        r_k = find_center_r_k(vertex_set)

        for (source, face_set) in enumerate(faces_list)
                        
            for (S_f, face) in enumerate(face_set)
                
                # Calculate the normal vector on the plane
                n_f = calculate_normal!(w1,face)

                for i in 1:3
                    for j in 1:3
                        C_subblock[i,j] -= 1.0*(i+j)
                    end 
                end
            end

            C[3 * (ele - 1) + 1:3 * ele, 3 * (source - 1) + 1:3 * source] = C_subblock
        end
    end

    return C
end

blank_C_matrix (generic function with 1 method)

In [50]:
length_box = 1.0
width = 1.0
height = 1.0

num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = blank_C_matrix(vertices_list, faces_list)

num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = blank_C_matrix(vertices_list, faces_list)

  1.904 μs (54 allocations: 4.59 KiB)
  43.334 μs (1206 allocations: 101.42 KiB)


30×30 Matrix{Float64}:
   -24.0    -36.0    -48.0    -48.0  …   -432.0   -240.0   -360.0   -480.0
   -36.0    -48.0    -60.0    -72.0      -540.0   -360.0   -480.0   -600.0
   -48.0    -60.0    -72.0    -96.0      -648.0   -480.0   -600.0   -720.0
  -264.0   -396.0   -528.0   -288.0      -912.0   -480.0   -720.0   -960.0
  -396.0   -528.0   -660.0   -432.0     -1140.0   -720.0   -960.0  -1200.0
  -528.0   -660.0   -792.0   -576.0  …  -1368.0   -960.0  -1200.0  -1440.0
  -504.0   -756.0  -1008.0   -528.0     -1392.0   -720.0  -1080.0  -1440.0
  -756.0  -1008.0  -1260.0   -792.0     -1740.0  -1080.0  -1440.0  -1800.0
 -1008.0  -1260.0  -1512.0  -1056.0     -2088.0  -1440.0  -1800.0  -2160.0
  -744.0  -1116.0  -1488.0   -768.0     -1872.0   -960.0  -1440.0  -1920.0
 -1116.0  -1488.0  -1860.0  -1152.0  …  -2340.0  -1440.0  -1920.0  -2400.0
 -1488.0  -1860.0  -2232.0  -1536.0     -2808.0  -1920.0  -2400.0  -2880.0
  -984.0  -1476.0  -1968.0  -1008.0     -2352.0  -1200.0  -1800.0  -2400.0
  

## Section 7: More Functions 

In [5]:
function create_B_app(vertices_list, direction, magnitude)
    N = length(vertices_list)
    B_app = zeros(3*N)
    
    for n in 1:N
        B_app[(n-1)*3+1:n*3] .= magnitude * direction
    end

    return B_app
end

function create_AA_chi(vertices_list, chi, C)
    N = length(vertices_list)
    A_chi = I(3*N) - chi / (1.25663706212e-6 * (1 + chi)) * C
    return A_chi
end

function find_M(A_chi, chi, B_app)
    M = A_chi \ (chi / (1.25663706212e-6 * (1 + chi)) * B_app)
    return M
end




find_M (generic function with 1 method)

In [6]:
length_box = 1
width = 1
height = 1

num_boxes_x = 10
num_boxes_y = 1
num_boxes_z = 1

vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
C = build_C_matrix(vertices_list, faces_list)

direction_B_0 = [1, 0, 0]
magnitude = 1e-5
chi = 100

B_app = create_B_app(vertices_list, direction_B_0, magnitude)
A_chi = create_A_chi(vertices_list, chi, C)
M = find_M(A_chi, chi, B_app)



LoadError: UndefVarError: SVector not defined

In [85]:
length_box = 1
width = 1
height = 1


for i in 1:7
    num_boxes_x = i
    num_boxes_y = i
    num_boxes_z = i

    total_boxes = num_boxes_x * num_boxes_y * num_boxes_z
    print("Iteration $i - Total number of boxes: $total_boxes")
    @time begin
        vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
        C = build_C_matrix(vertices_list, faces_list)
    end
    println()
    @time begin
        direction_B_0 = [1, 0, 0]
        magnitude = 1e-5
        chi = 100

        B_app = create_B_app(vertices_list, direction_B_0, magnitude)
        A_chi = create_A_chi(vertices_list, chi, C)
        M = find_M(A_chi, chi, B_app)
    end
    M
    println()
end

Iteration 1 - Total number of boxes: 1  0.000236 seconds (1.91 k allocations: 127.609 KiB)

  0.000014 seconds (9 allocations: 800 bytes)

Iteration 2 - Total number of boxes: 8  0.009613 seconds (120.06 k allocations: 7.810 MiB)

  0.000024 seconds (17 allocations: 15.656 KiB)

Iteration 3 - Total number of boxes: 27  0.108511 seconds (1.37 M allocations: 88.804 MiB, 6.57% gc time)

  0.000099 seconds (39 allocations: 159.281 KiB)

Iteration 4 - Total number of boxes: 64  0.610801 seconds (7.67 M allocations: 498.750 MiB, 4.89% gc time)

  0.009106 seconds (76 allocations: 875.844 KiB)

Iteration 5 - Total number of boxes: 125  2.352016 seconds (29.25 M allocations: 1.858 GiB, 5.10% gc time)

  0.004522 seconds (137 allocations: 3.241 MiB)

Iteration 6 - Total number of boxes: 216  7.216018 seconds (87.35 M allocations: 5.547 GiB, 5.07% gc time)

  0.009161 seconds (228 allocations: 9.648 MiB)

Iteration 7 - Total number of boxes: 343 17.785363 seconds (220.25 M allocations: 13.986 Gi